In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
from datetime import datetime

print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
from DateProvider import DateProvider
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
def getWorkingDirectory():
    pwd = ! "pwd"
    return pwd[0]

In [ ]:
pwd = getWorkingDirectory()

In [ ]:
pwd

## Download VAERS-Data

In [ ]:
import os
import time

class AndroidEmulator:
    
    @staticmethod
    def run(runnable):
        AndroidEmulator._start()
        result = runnable()
        AndroidEmulator._stop()
        return result
    
    @staticmethod
    def _start():
        os.system("/home/frankknoll/Android/Sdk/emulator/emulator -avd Pixel_2_API_30 -no-window &")
        AndroidEmulator._waitUntilStarted()
        
    @staticmethod
    def _waitUntilStarted():
        while not AndroidEmulator._isStarted():
            time.sleep(1)

    @staticmethod
    def _isStarted():
        boot_completed = ! adb shell getprop sys.boot_completed
        return boot_completed[0] == '1'

    @staticmethod
    def _stop():
        ! adb emu kill

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def _getOptions(downloadDir, isHeadless):
    options = Options()
    options.headless = isHeadless
    options.add_experimental_option("prefs", {"download.default_directory" : downloadDir})
    return options

def getWebDriver(downloadDir, isHeadless):
    return webdriver.Chrome(
        service = ChromeService(executable_path = ChromeDriverManager().install()),
        options = _getOptions(downloadDir, isHeadless))

def saveCaptchaImageAs(driver, captchaImageFile):
    captchaImage = driver.find_element(By.CSS_SELECTOR, "img[src='captchaImage']")
    with open(captchaImageFile, 'wb') as file:
        file.write(captchaImage.screenshot_as_png)

def existsElementWithId(driver, id):
    return len(driver.find_elements(By.ID, id)) > 0

def isCaptchaSolved(driver):
    return not existsElementWithId(driver, "wordverify")


In [ ]:
import time
import os

def waitUntilDownloadHasFinished(file):
    while not os.path.exists(file):
        time.sleep(2)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
import numpy as np
import io

# copied from value of characters variable in captcha_ocr.ipynb or captcha_ocr_trainAndSaveModel.ipynb
characters = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'a', 'b', 'c', 'd', 'e', 'f']

img_width = 241
img_height = 62

downsample_factor = 4

# copied from value of max_length variable in captcha_ocr.ipynb or captcha_ocr_trainAndSaveModel.ipynb
max_length = 6

char_to_num = layers.StringLookup(
    vocabulary=list(characters),
    mask_token=None)

num_to_char = layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(),
    mask_token=None, invert=True)

def encode_single_sample(img_path):
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_height, img_width])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])
    # 7. Return a dict as our model is expecting two inputs
    return asSingleSampleBatch(img)

def asSingleSampleBatch(img):
    array = keras.utils.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][:, :max_length]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text

def load_model():
    _model = keras.models.load_model('model')
    model = keras.models.Model(
        _model.get_layer(name="image").input,
        _model.get_layer(name="dense2").output)
    return model

def getTextInCaptchaImage(captchaImageFile):
    batchImages = encode_single_sample(captchaImageFile)
    preds = model.predict(batchImages)
    return decode_batch_predictions(preds)[0]

print("loading model...")
model = load_model()
model.summary()

In [ ]:
import os

#def getTextInCaptchaImage(captchaImageFile):
#    baseDir = "~/AndroidStudioProjects/TextRecognizer"
#    ! cp $captchaImageFile $baseDir/app/src/main/assets/captchas/captcha_image.jpeg
#    ! cd $baseDir;./gradlew connectedAndroidTest
#    textInCaptchaImage = ! adb shell "run-as org.textrecognizer cat /data/data/org.textrecognizer/files/captcha_image.txt"
#    return textInCaptchaImage[0]
    
def solveCaptchaAndStartFileDownload(driver, captchaImageFile):
    saveCaptchaImageAs(driver, captchaImageFile)
    textInCaptchaImage = getTextInCaptchaImage(captchaImageFile)
    display('textInCaptchaImage: ', textInCaptchaImage)
    driver.find_element(By.ID, "verificationCode").send_keys(textInCaptchaImage)
    driver.find_element(By.CSS_SELECTOR, '[name="downloadbut"]').click()

def downloadFile(absoluteFile, driver, maxTries):
    def _downloadFile():
        driver.get('https://vaers.hhs.gov/eSubDownload/index.jsp?fn=' + os.path.basename(absoluteFile))
        solveCaptchaAndStartFileDownload(driver, 'captchaImage.jpeg')

    numTries = 1
    _downloadFile()
    while(not isCaptchaSolved(driver) and (maxTries is None or numTries < maxTries)):
        _downloadFile()
        numTries = numTries + 1

    if isCaptchaSolved(driver):
        waitUntilDownloadHasFinished(absoluteFile)
        return absoluteFile
    else:
        return None

def downloadVAERSFile(file, downloadDir):
    driver = getWebDriver(downloadDir, isHeadless = True)
    downloadedFile = downloadFile(
        absoluteFile = downloadDir + "/" + file,
        driver = driver,
        maxTries = None)
    driver.quit()
    return downloadedFile

In [ ]:
import zipfile
import os

def unzip(zipFile, dstDir):
    with zipfile.ZipFile(zipFile, 'r') as zip_ref:
        zip_ref.extractall(dstDir)

def unzipAndRemove(zipFile, dstDir):
    unzip(zipFile, dstDir)
    os.remove(zipFile)

def downloadVAERSFileAndUnzip(file):
    downloadedFile = downloadVAERSFile(file, getWorkingDirectory() + "/VAERS/tmp")
    unzipAndRemove(
        zipFile = downloadedFile,
        dstDir = getWorkingDirectory() + '/VAERS/')


In [ ]:
if needsUpdate:
    downloadVAERSFileAndUnzip('2022VAERSData.zip')
    downloadVAERSFileAndUnzip('NonDomesticVAERSData.zip')

In [ ]:
from VaersDescrReader import VaersDescrReader


In [ ]:
from VaersDescr2DataFrameConverter import VaersDescr2DataFrameConverter

In [ ]:
from DataFrameNormalizer import DataFrameNormalizer

In [ ]:
from DataFrameFilter import DataFrameFilter

In [ ]:
from CountryColumnAdder import CountryColumnAdder

In [ ]:
from SevereColumnAdder import SevereColumnAdder

In [ ]:
from BatchCodeTableFactory import BatchCodeTableFactory

In [ ]:
from HtmlTransformerUtil import HtmlTransformerUtil

In [ ]:
from CountryOptionsSetter import CountryOptionsSetter

In [ ]:
from bs4 import BeautifulSoup


def saveCountryOptions(countryOptions):
    HtmlTransformerUtil().applySoupTransformerToFile(
        file = "../docs/batchCodeTable.html",
        soupTransformer =
            lambda soup:
                BeautifulSoup(
                    CountryOptionsSetter().setCountryOptions(html = str(soup), options = countryOptions),
                    'lxml'))


In [ ]:
def saveLastUpdatedBatchCodeTable(lastUpdated):
    def setLastUpdated(soup):
        soup.find(id = "last_updated").string.replace_with(lastUpdated.strftime(DateProvider.DATE_FORMAT))
        return soup

    HtmlTransformerUtil().applySoupTransformerToFile(
        file = "../docs/batchCodeTable.html",
        soupTransformer = setLastUpdated)

In [ ]:
from IOUtils import IOUtils

In [ ]:
def getVaersForYears(years):
    def addCountryColumn(dataFrame):
        dataFrame['COUNTRY'] = 'United States'
        return dataFrame

    return _getVaers(
        _getVaersDescrReader().readVaersDescrsForYears(years),
        addCountryColumn)

def getNonDomesticVaers():
    return _getVaers(
        [_getVaersDescrReader().readNonDomesticVaersDescr()],
        CountryColumnAdder.addCountryColumn)

def _getVaersDescrReader():
    return VaersDescrReader(dataDir = "VAERS")

def _getVaers(vaersDescrs, addCountryColumn):
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    dataFrame = addCountryColumn(dataFrame)
    DataFrameNormalizer.normalize(dataFrame)
    dataFrame = SevereColumnAdder.addSevereColumn(dataFrame)
    return dataFrame

In [ ]:
internationalVaers = pd.concat([getVaersForYears([2020, 2021, 2022]), getNonDomesticVaers()])
internationalVaersCovid19 = DataFrameFilter().filterByCovid19(internationalVaers)
internationalVaersCovid19

In [ ]:
def createAndSaveBatchCodeTableForCountry(createBatchCodeTableForCountry, country, minADRsForLethality = None):
    batchCodeTable = createBatchCodeTableForCountry(country)
    batchCodeTable.index.set_names("Batch", inplace = True)
    if minADRsForLethality is not None:
        batchCodeTable.loc[batchCodeTable['Adverse Reaction Reports'] < minADRsForLethality, ['Severe reports', 'Lethality']] = [np.nan, np.nan]
    IOUtils.saveDataFrame(batchCodeTable, '../docs/data/batchCodeTables/' + country)
    # display(country + ":", batchCodeTable)
    display(country)

def createAndSaveBatchCodeTablesForCountries(createBatchCodeTableForCountry, countries, minADRsForLethality = None):
    for country in countries:
        createAndSaveBatchCodeTableForCountry(createBatchCodeTableForCountry, country, minADRsForLethality)

In [ ]:
def getCountryOptions(countries):
    return [getCountryOption(country) for country in countries]

def getCountryOption(country):
    return '<option value="{country}">{country}</option>'.format(country = country)

In [ ]:
countries = sorted(internationalVaersCovid19['COUNTRY'].unique())
countryOptions = ['<option value="Global" selected>Global</option>']  + getCountryOptions(countries)

In [ ]:
saveCountryOptions(countryOptions)

In [ ]:
saveLastUpdatedBatchCodeTable(dateProvider.getLastUpdatedDataSource())

In [ ]:
minADRsForLethality = 100
batchCodeTableFactory = BatchCodeTableFactory(internationalVaersCovid19)

createAndSaveBatchCodeTablesForCountries(
    createBatchCodeTableForCountry = lambda country: batchCodeTableFactory.createBatchCodeTableByCountry(country),
    countries = countries,
    minADRsForLethality = minADRsForLethality)

createAndSaveBatchCodeTableForCountry(
    createBatchCodeTableForCountry = lambda country: batchCodeTableFactory.createGlobalBatchCodeTable(),
    country = 'Global',
    minADRsForLethality = minADRsForLethality)

In [ ]:
def publishGitHubPages():
    %cd /home/frankknoll/Dokumente/Corona/projects/HowBadIsMyBatch-pages
    ! git add -A
    ! git commit -m "updating data for batch codes"
    ! git push

### see https://knollfrank.github.io/HowBadIsMyBatch/batchCodeTable.html

In [ ]:
publishGitHubPages()